In [11]:
import os
import sys
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

import matplotlib.pyplot as plt
from tqdm import tqdm

current_dir = os.getcwd()
sys.path.append(os.path.join(current_dir, ".."))

from scripts.logger import LoggerManager
from scripts.generaldataset import UNSWNB15


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
lm = LoggerManager(log_dir=f"{current_dir}/logs", log_name="test_dl_models")
lm.logger.info("Logger initialized")

dataset = UNSWNB15(logger=lm.logger).encode(attack_encoder="onehot").scale(scaler="minmax").optimize_memory()

2025-11-16 23:00:08,220 - INFO - Logger initialized
2025-11-16 23:00:08,222 - INFO - Downloading dataset: mrwellsdavid/unsw-nb15
2025-11-16 23:00:08,820 - ERROR - File not found: Dataset file not found at: C:\Users\valen\.cache\kagglehub\datasets\mrwellsdavid\unsw-nb15\versions\1\UNSW-NB15.csv


FileNotFoundError: Dataset file not found at: C:\Users\valen\.cache\kagglehub\datasets\mrwellsdavid\unsw-nb15\versions\1\UNSW-NB15.csv